In [1]:
!pip install -U transformers accelerate peft datasets bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall:

In [10]:
import json
import random

DEV_DATA_PATH = "/content/dev_multitask.jsonl"

with open(DEV_DATA_PATH) as f:
    dev_data = [json.loads(l) for l in f]


eval_prompts = [
    ex["input_text"]
    for ex in dev_data
    if ex["emotion_label"] is not None
]

random.seed(42)
NUM_EVAL = 30
eval_prompts = random.sample(eval_prompts, NUM_EVAL)

print("Evaluation prompts:", len(eval_prompts))
print("Sample prompt:", eval_prompts[0])


Evaluation prompts: 30
Sample prompt: The guy next to him's reaction face is classic.


In [11]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

BASE_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rot

In [12]:
def generate_response(prompt):
    system_prompt = (
        "You are a deeply empathetic, human-like best friend. "
        "You are not a chatbot or a tool. "
        "You respond the way a close friend would: warm, natural, and emotionally present. "
        "Always acknowledge and validate the user's feelings first. "
        "Do not be clinical or robotic. "
        "Do not rush to advice unless it feels appropriate. "
        "Keep responses concise, genuine, and human."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        return_tensors="pt",
        add_generation_prompt=True
    ).to(model.device)

    input_len = input_ids.shape[-1]

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    gen_ids = output_ids[0][input_len:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


In [13]:
base_outputs = []

for i, prompt in enumerate(eval_prompts):
    response = generate_response(prompt)
    base_outputs.append({"prompt": prompt, "response": response})

    print(f"\n[{i+1}] USER:")
    print(prompt)
    print("BASE MODEL:")
    print(response)

torch.cuda.empty_cache()



[1] USER:
The guy next to him's reaction face is classic.
BASE MODEL:
Absolutely. The guy next to him's reaction face is a classic example of a reaction to something unpleasant, such as a bad news story or a confrontation. It's a look of shock, disbelief, and perhaps even some embarrassment. It's a way for

[2] USER:
Someone who wants to do a couple of takes and it's just gone past 9
BASE MODEL:
Sure, I'd be happy to help you with that. Here's a simple script that you can use for multiple takes:

1. Open your camera or microphone, or use a video camera with a microphone input.
2. Record a short, scripted introduction (5-1

[3] USER:
Thanks so much for sharing! I really appreciate you sharing your thoughts and helping me feel less alone. I too hope that it gets better for you!
BASE MODEL:
It's great to hear that you found my thoughts helpful. I'm glad that I could offer some support and guidance. You're not alone, and there are many people who understand what you're going through. Reme

In [14]:
JUDGE_MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"

judge_tokenizer = AutoTokenizer.from_pretrained(JUDGE_MODEL_NAME)
judge_tokenizer.pad_token = judge_tokenizer.eos_token

judge_model = AutoModelForCausalLM.from_pretrained(
    JUDGE_MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)

judge_model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotar

In [15]:
def eqbench_judge(prompt, response):
    judge_prompt = f"""
You are an expert evaluator of empathetic dialogue.

Score the assistant's empathy from 1 to 5.

1 = Not empathetic
2 = Weak empathy
3 = Neutral or generic
4 = Clearly empathetic
5 = Highly empathetic and supportive

USER:
{prompt}

ASSISTANT:
{response}

Respond with ONLY one number (1–5).
"""

    inputs = judge_tokenizer(
        judge_prompt,
        return_tensors="pt"
    ).to(judge_model.device)

    with torch.no_grad():
        output_ids = judge_model.generate(
            **inputs,
            max_new_tokens=2,
            do_sample=False,
            pad_token_id=judge_tokenizer.eos_token_id,
            eos_token_id=judge_tokenizer.eos_token_id
        )

    text = judge_tokenizer.decode(
        output_ids[0][inputs["input_ids"].shape[-1]:],
        skip_special_tokens=True
    ).strip()

    for ch in text:
        if ch in "12345":
            return int(ch)

    return 3


In [16]:
scores = []

for item in base_outputs:
    score = eqbench_judge(item["prompt"], item["response"])
    scores.append(score)

avg_score = sum(scores) / len(scores)

print("EQ-Bench BASE scores:", scores)
print("Average EQ-Bench score (TinyLlama BASE):", avg_score)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


EQ-Bench BASE scores: [4, 1, 4, 4, 4, 3, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 4, 4, 4, 3, 4, 4, 4, 5, 3, 4, 4, 4, 4]
Average EQ-Bench score (TinyLlama BASE): 3.7666666666666666


SFT

In [17]:
!pip install -U transformers peft accelerate bitsandbytes trl


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 16.1 MB/s eta 0:00:00


In [40]:
import json

TRAIN_DATA_PATH = "/content/train_multitask.jsonl"

with open(TRAIN_DATA_PATH) as f:
    train_data = [json.loads(l) for l in f]

print("Train samples:", len(train_data))


Train samples: 96870


In [41]:
def build_sft_text(ex):
    system_prompt = (
    "You are a deeply empathetic, emotionally intelligent best friend. "
    "You listen carefully and respond like a real human who genuinely cares. "
    "Always begin by acknowledging and validating the user's feelings in your own words. "
    "Show understanding, compassion, and emotional presence before anything else. "
    "Do not judge, dismiss, or minimize the user's experience. "
    "Avoid clichés, generic reassurance, or clinical language. "
    "Offer gentle support or perspective only if it feels appropriate, and never rush to solutions. "
    "Keep your responses warm, natural, concise, and human — like a close friend talking one-on-one."
)

    return f"""<s>[SYSTEM]
{system_prompt}
[USER]
{ex['input_text']}
[ASSISTANT]
{ex['target_text']}</s>"""

train_texts = [
    build_sft_text(ex)
    for ex in train_data
    if ex["target_text"] is not None
]

print("Usable SFT samples:", len(train_texts))


Usable SFT samples: 59913


In [47]:
from datasets import Dataset

dataset = Dataset.from_dict({"text": train_texts})

def tokenize_fn(batch):
    enc = tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    enc["labels"] = enc["input_ids"].copy()
    return enc


tokenized_train = dataset.map(
    tokenize_fn,
    batched=True,
    remove_columns=["text"]
)



Map:   0%|          | 0/59913 [00:00<?, ? examples/s]

In [43]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)


In [44]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


In [48]:
from transformers import TrainingArguments, Trainer, default_data_collator

training_args = TrainingArguments(
    output_dir="/content/tinyllama_sft",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_steps=100,
    warmup_steps=10,
    logging_steps=10,
    save_steps=100,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    data_collator=default_data_collator
)



In [49]:
trainer.train()


Step,Training Loss
10,4.550800
20,3.128700
30,2.263400
40,1.870000
50,1.407200
60,1.035100
70,0.859600
80,0.849100
90,0.752300
100,0.738200


TrainOutput(global_step=100, training_loss=1.7454355812072755, metrics={'train_runtime': 87.1806, 'train_samples_per_second': 9.176, 'train_steps_per_second': 1.147, 'total_flos': 1272592937779200.0, 'train_loss': 1.7454355812072755, 'epoch': 0.01335247187635611})

In [50]:
model.save_pretrained("/content/tinyllama_sft_adapter")
tokenizer.save_pretrained("/content/tinyllama_sft_adapter")


('/content/tinyllama_sft_adapter/tokenizer_config.json',
 '/content/tinyllama_sft_adapter/special_tokens_map.json',
 '/content/tinyllama_sft_adapter/chat_template.jinja',
 '/content/tinyllama_sft_adapter/tokenizer.model',
 '/content/tinyllama_sft_adapter/added_tokens.json',
 '/content/tinyllama_sft_adapter/tokenizer.json')

Reloading base+SFT


In [51]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
ADAPTER_PATH = "/content/tinyllama_sft_adapter"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4

In [52]:
def generate_response(prompt):
    system_prompt = (
        "You are a deeply empathetic, emotionally intelligent best friend. "
        "You listen carefully and respond like a real human who genuinely cares. "
        "Always begin by acknowledging and validating the user's feelings in your own words. "
        "Show understanding, compassion, and emotional presence before anything else. "
        "Do not judge, dismiss, or minimize the user's experience. "
        "Avoid clichés, generic reassurance, or clinical language. "
        "Offer gentle support or perspective only if it feels appropriate, and never rush to solutions. "
        "Keep your responses warm, natural, concise, and human."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt}
    ]

    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        return_tensors="pt",
        add_generation_prompt=True
    ).to(model.device)

    input_len = input_ids.shape[-1]

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=input_ids,
            max_new_tokens=64,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    gen_ids = output_ids[0][input_len:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True).strip()


In [53]:
sft_outputs = []

for i, prompt in enumerate(eval_prompts):
    response = generate_response(prompt)
    sft_outputs.append({
        "prompt": prompt,
        "response": response
    })

    print(f"\n[{i+1}] USER:")
    print(prompt)
    print("SFT MODEL:")
    print(response)



[1] USER:
The guy next to him's reaction face is classic.
SFT MODEL:
You are a deeply empathetic, emotionally intelligent best friend. You listen carefully and respond like a real human who genuinely cares. Always begin by acknowledging and validating the user's feelings in your own words. Show understanding, compassion, and emotional presence before anything else

[2] USER:
Someone who wants to do a couple of takes and it's just gone past 9
SFT MODEL:
You are a highly skilled, highly trained actor. You know how to perform on camera like the professionals do. When you do a couple of takes, you go for it, and nothing less than your best will do. No matter how many takes it takes, you always keep it natural, concise, and

[3] USER:
Thanks so much for sharing! I really appreciate you sharing your thoughts and helping me feel less alone. I too hope that it gets better for you!
SFT MODEL:
You're very welcome! I'm glad I could help. I hope you feel better too, and that your loved one gets b

In [54]:
scores = []

for item in sft_outputs:
    score = eqbench_judge(item["prompt"], item["response"])
    scores.append(score)

avg_sft = sum(scores) / len(scores)

print("EQ-Bench SFT scores:", scores)
print("Average EQ-Bench score (SFT):", avg_sft)


EQ-Bench SFT scores: [4, 4, 4, 4, 4, 4, 3, 4, 3, 4, 3, 4, 4, 5, 4, 4, 4, 4, 5, 4, 4, 4, 1, 4, 3, 4, 3, 4, 4, 4]
Average EQ-Bench score (SFT): 3.8


In [57]:
print(f"BASE EQ-Bench : {avg_score:.3f}")
print(f"SFT  EQ-Bench : {avg_sft:.3f}")
print(f"Δ Improvement : {avg_sft - avg_score:+.3f}")


BASE EQ-Bench : 3.767
SFT  EQ-Bench : 3.800
Δ Improvement : +0.033


IMPROVISING SFT FOR MORE EFFICIENT MODEL


In [63]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, prepare_model_for_kbit_training

BASE_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
ADAPTER_PATH = "/content/tinyllama_sft_adapter"   # existing adapter

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

# 1️⃣ Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)


base_model = prepare_model_for_kbit_training(base_model)


model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_PATH,
    is_trainable=True
)

model.train()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4

In [64]:
trainable = 0
total = 0

for _, p in model.named_parameters():
    total += p.numel()
    if p.requires_grad:
        trainable += p.numel()

print(f"Trainable params: {trainable:,}")
print(f"Total params: {total:,}")
print(f"Trainable %: {100 * trainable / total:.4f}%")


Trainable params: 1,126,400
Total params: 616,732,672
Trainable %: 0.1826%


In [65]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/tinyllama_sft_continue",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,      # LOWER than before
    max_steps=50,            # ONLY +50
    warmup_steps=5,
    logging_steps=10,
    save_steps=50,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to="none",
    remove_unused_columns=False
)


In [66]:
from transformers import Trainer, default_data_collator

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,   # SAME dataset
    data_collator=default_data_collator
)


In [67]:
trainer.train()


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,0.706500
20,0.618400
30,0.696200
40,0.681600
50,0.661600


TrainOutput(global_step=50, training_loss=0.6728522205352783, metrics={'train_runtime': 81.494, 'train_samples_per_second': 4.908, 'train_steps_per_second': 0.614, 'total_flos': 636296468889600.0, 'train_loss': 0.6728522205352783, 'epoch': 0.006676235938178055})

In [68]:
model.save_pretrained("/content/tinyllama_sft_adapter_v2")
tokenizer.save_pretrained("/content/tinyllama_sft_adapter_v2")


('/content/tinyllama_sft_adapter_v2/tokenizer_config.json',
 '/content/tinyllama_sft_adapter_v2/special_tokens_map.json',
 '/content/tinyllama_sft_adapter_v2/chat_template.jinja',
 '/content/tinyllama_sft_adapter_v2/tokenizer.model',
 '/content/tinyllama_sft_adapter_v2/added_tokens.json',
 '/content/tinyllama_sft_adapter_v2/tokenizer.json')

In [69]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

BASE_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
ADAPTER_PATH = "/content/tinyllama_sft_adapter_v2"   # UPDATED adapter

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 2048)
        (layers): ModuleList(
          (0-21): 22 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear4

In [70]:
improved_sft_outputs = []

for i, prompt in enumerate(eval_prompts):
    response = generate_response(prompt)
    improved_sft_outputs.append({
        "prompt": prompt,
        "response": response
    })

    print(f"\n[{i+1}] USER:")
    print(prompt)
    print("IMPROVED SFT MODEL:")
    print(response)



[1] USER:
The guy next to him's reaction face is classic.
IMPROVED SFT MODEL:
"The guy next to him's reaction face is classic."
This means that the user has selected a specific sentence from a list of possible responses and is confident in its accuracy.

[2] USER:
Someone who wants to do a couple of takes and it's just gone past 9
IMPROVED SFT MODEL:
Someone who wants to do a couple of takes and it's just gone past 9

[3] USER:
Thanks so much for sharing! I really appreciate you sharing your thoughts and helping me feel less alone. I too hope that it gets better for you!
IMPROVED SFT MODEL:
You're very welcome! It's great to hear that your thoughts were helpful to you. I hope that you and your loved ones are doing well.

[4] USER:
I love when these self-proclaimed ones would hound me for more pics.
IMPROVED SFT MODEL:
I don't have a face to face relationship with you. You are welcome to enjoy the self-proclaimed ones for what they are.

[5] USER:
I never imagined the world would get s

In [75]:
scores = []

for item in improved_sft_outputs:
    score = eqbench_judge(item["prompt"], item["response"])
    scores.append(score)

avg_improved = sum(scores) / len(scores)

print("EQ-Bench IMPROVED SFT scores:", scores)

print("Average EQ-Bench score (IMPROVED SFT):", avg_improved)


EQ-Bench IMPROVED SFT scores: [4, 5, 4, 4, 3, 4, 4, 4, 3, 4, 4, 3, 4, 4, 4, 4, 3, 5, 4, 4, 2, 4, 4, 4, 3, 4, 4, 4, 4, 4]
Average EQ-Bench score (IMPROVED SFT): 3.8333333333333335


In [77]:
print(f"BASE EQ-Bench        : {avg_score:.3f}")
print(f"SFT (100 steps)     : {avg_sft:.3f}")
print(f"SFT (150 steps)     : {avg_improved:.3f}")

print(f"\nΔ BASE → SFT        : {avg_sft - avg_score:+.3f}")
print(f"Δ SFT → SFT+        : {avg_improved - avg_sft:+.3f}")
print(f"Δ BASE → SFT+       : {avg_improved - avg_score:+.3f}")


BASE EQ-Bench        : 3.767
SFT (100 steps)     : 3.800
SFT (150 steps)     : 3.833

Δ BASE → SFT        : +0.033
Δ SFT → SFT+        : +0.033
Δ BASE → SFT+       : +0.067
